# NLP Answers

- **Answer Set**: No. 06
- **Full Name**: Mohammad Hosein Nemati
- **Student Code**: `610300185`

---

## Introduction

In this problem, we are going to use **ParsBERT** pre-trained model for **Token Classification (NER)** task on **Custom** dataset.  
Then we will compare the reported metrics to previously trained models in `exercise 3`

In the first step, we will import some useful libraries.

In [6]:
import warnings

import math as math
import hazm as hazm
import nltk as nltk
import nltk.corpus.reader.conll as nltkconll

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn.preprocessing as skprocessing

warnings.filterwarnings("ignore", category=UserWarning)

---

## Loading

First of all, we must load our dataset and then shuffle the records.

In [2]:
train_reader = nltkconll.ConllCorpusReader("../lib", ["Train.txt"], ("words", "pos"))
test_reader = nltkconll.ConllCorpusReader("../lib", ["Test.txt"], ("words", "pos"))

words_frequency = nltk.FreqDist(train_reader.words())

def sents(reader):
    for sent in reader.sents():
        yield [
            word if (words_frequency.get(word) or 0) > 1 else "OOV"
            for word in sent
        ]

def words(reader):
    for word in reader.words():
        if (words_frequency.get(word) or 0) > 1:
            yield word
        else:
            yield "OOV"

def tags(reader):
    for sent in reader.tagged_sents():
        for token in sent:
            yield token[1]

In [25]:
from datasets import load_dataset

load_dataset("conll2003", data_files={
    "train": "../lib/Train.txt",
    "test": "../lib/Test.txt",
})

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'https:\\huggingface.co\\datasets\\conll2003\\resolve\\main'

---

## Preprocessing

In [7]:
word_encoder = skprocessing.LabelEncoder().fit(list(words(train_reader)))
tag_encoder = skprocessing.LabelEncoder().fit(list(tags(train_reader)))

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-base-uncased")

  0%|          | 0/11 [00:00<?, ?ba/s]

In [ ]:
train_tokens = tokenizer()

---

## Training

In [12]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained("HooshvareLab/bert-fa-base-uncased", num_labels=len(tag_encoder.classes_))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.fit(train_set, epochs=4)

Epoch 1/4
1100/1100 [==============================] - 1132s 1s/step - loss: 0.7946 - accuracy: 0.7883
Epoch 2/4
1100/1100 [==============================] - 1127s 1s/step - loss: 0.2830 - accuracy: 0.9215
Epoch 3/4
1100/1100 [==============================] - 1127s 1s/step - loss: 0.1694 - accuracy: 0.9545
Epoch 4/4
1100/1100 [==============================] - 1126s 1s/step - loss: 0.1043 - accuracy: 0.9723


---

## Testing

In [13]:
result = model.evaluate([1,2])

print(result)

AttributeError: in user code:

    File "c:\Users\KoLiBer\Documents\Workspace\nlpexercises\.venv\lib\site-packages\keras\engine\training.py", line 1557, in test_function  *
        return step_function(self, iterator)
    File "c:\Users\KoLiBer\Documents\Workspace\nlpexercises\.venv\lib\site-packages\keras\engine\training.py", line 1546, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\KoLiBer\Documents\Workspace\nlpexercises\.venv\lib\site-packages\keras\engine\training.py", line 1535, in run_step  **
        outputs = model.test_step(data)
    File "c:\Users\KoLiBer\Documents\Workspace\nlpexercises\.venv\lib\site-packages\transformers\modeling_tf_utils.py", line 1492, in test_step
        y = {key: val for key, val in x.items() if key in label_kwargs}

    AttributeError: 'Tensor' object has no attribute 'items'


As we can see, the **Pars BERT** pre-trained model returend much better accuracy after fine-tuning.

---